Do some imports if not already performed:

In [ ]:
try:
    startup_was_launched
except: 
    %run -i "..//tools//startup.ipynb"
    print('Startup performed ...')

In [ ]:
progress = progress_bar(total=10)

# 1. INSTRUMENT IMPORT

### Neel DAC

In [ ]:
from tools.drivers.NEEL_DAC import NEEL_DAC
# Load NEEL_DAC
dac = NEEL_DAC(name='dac',
            bitFilePath = '..\\tools\\drivers\\fpgabatchhewodan_sbRIO9612RIO0_hewodan_kUFBPXPrLOs.lvbitx',
            address = '192.168.0.3',
            used_buses = [1,2,4,6], # on Wodan only these slots are used
            ms2wait=1)

# Get current DAC values from FPGA and store it to sub modules
dac.get_DAC_values()

station.add_component(dac)

In [ ]:
progress.update(1)
progress.set_description('{:>40s}'.format('Loading Neel DAC.'))
progress.refresh() # to show immediately the update

In [ ]:
# Initialize DAC
dac.DAC_init_values()
dac.metadata['comments'] = 'All DAC values are initialised to 0 V.'
progress.update(1)
progress.set_description('{:>40s}'.format('Moving Neel DAC values to zero.'))
progress.refresh() # to show immediately the update

### NI ADC

In [ ]:
from tools.drivers.NI6216 import NI6216
adc = NI6216(name = 'adc',
             ai_chans = [0,1,2,3,4,5,6,7],
             ao_chans = [0,1],
             ai_sample_rate = 250000,
             trigger_source = 'PFI0',
             trigger_mode = 'free',
             trigger_edge = 'fall',
             average_time = 100,
             acquire_points = 101,
             terminal_config = 'Differential',
             voltage_range = ['+-0.2V', '+-1V', '+-5V', '+-10V'][3],
             )
station.add_component(adc)
adc.measure() # to update values
adc.metadata['comments'] = 'AI0: DC readout<br />AI1: lock-in current readout<br />AI2: lock-in phase readout<br />AO0: magnetic field control'

In [ ]:
progress.update(1)
progress.set_description('{:>40s}'.format('Loading NI ADC card.'))
progress.refresh() # to show immediately the update

### SR7265 Lock-in amplifier

In [ ]:
from tools.drivers.SR7265 import SR7265
lockins = []
gpib_addrs = [12,13,14]
for ind,gpib_addr in enumerate(gpib_addrs):
    li_name = 'lockin_{:d}'.format(ind)
    li_addr = 'GPIB::{:d}::0::INSTR'.format(gpib_addr)
    lockins.append(SR7265(name = li_name,address = li_addr))
    lockins[ind].metadata['comments'] = 'Use via list entry lockins[{:d}]'.format(ind)
    station.add_component(lockins[ind])
    

In [ ]:
# Set properties of lockins:
for lockin in lockins:
    lockin.sensitivity(0.2) # V
    lockin.time_constant(0.1) # sec
    message = 'Set {:s} to sensitivity of {:f} {:s} and time constant of {:f} {:s}.'
    message_args = [lockin.name,
                    lockin.sensitivity(),
                    lockin.sensitivity.unit,
                    lockin.time_constant(),
                    lockin.time_constant.unit]
    print(message.format(*message_args))

In [ ]:
progress.update(1)
progress.set_description('{:>40s}'.format('Loading SR Lockin amplifier.'))
progress.refresh() # to show immediately the update

### Other instruments
Import other instruments here ...

In [ ]:
progress.update(1)
progress.set_description('{:>40s}'.format('Loading other instruments.'))
progress.refresh() # to show immediately the update

# 2. EXPERIMENT CONTROLS
The instrument parameters that control your experiment are bundled in the abstract "controls" instrument that is created at startup:

In [ ]:
list_instruments(station)

Now add the intrument parameters that you use to control your experiment with the syntax:
```Python
ref = controls.add('ref',
            source = instrument_parameter_value,
            # Adapt the parameter according to your exp. setup:
            label = r'Reasonable label $r_{\rm L}$',
            # ...
            )
```

In [ ]:
progress.update(1)
progress.set_description('{:>40s}'.format('Setting-up experiment controller.'))
progress.refresh() # to show immediately the update

### Example gate voltages

In [ ]:
# Control of quantum point contact (QPC):
qpc = controls.add('qpc',
            source = dac.p1.c0.v,
            move_command = dac.move,
            label  = r'Quantum point contact $V_{\rm QPC}$',
            unit   = 'V',
            vals   = vals.Numbers(-2.5,0.3), # limits
            )

# Control of tunnel barrier
barrier = controls.add('barrier',
            source = dac.p2.c0.v,
            label  = r'Tunnel barrier $V_{\rm B}$',
            unit   = 'V',
            vals = vals.Numbers(-2.5,0.3),
            )

# ... 

In [ ]:
progress.update(1)

### Example DC bias pinch-off gate and readout via IV-converter

In [ ]:
# DC bias with voltage divider:
dc_bias = controls.add('dc_bias',
            source = dac.p4.c0.v,
            label  = r'Voltage bias $V_{\rm Bias}$',
            unit   = 'V',
            scale  = 500, # ratio of dac voltage to value on sample; here 500 due to 1/500 voltage divider
            vals   = vals.Numbers(-1e-3,1e-3), # in user units (value on sample)
            )

# Readout via IV-converter:
i_qpc = controls.add('i_qpc',
            source =adc.ai0.v,
            label  = r'QPC current $I_{\rm QPC}$',
            unit   = 'A',
            scale  = 1e5, # ratio of dac voltage to current through QPC; here amplification factor of IV-converter
            )

In [ ]:
progress.update(1)

### Example DAC-lock-in bias and Lock-in readout via SR7265

In [ ]:
ac_freq = controls.add('ac_freq',source=dac.LI_frequency) # frequency in Hz
ac_ampl = controls.add('ac_ampl',
            source = dac.LI_amplitude,
            scale  = 500,                   # let's assume we have a 1/500 divider for the AC signal
            vals   = vals.Numbers(0.0,1e-3) # let's limit the amplitude of the AC signal to 1 mV
            )
ac_chan = controls.add('ac_chan',source=dac.LI_channel) # List [panel_id, channel_id] starting from respectively from 0
ac_bias = controls.add('ac_bias',source=dac.LI_status) # Boolean to turn on/off generation of lock-in signal

In [ ]:
# Magnitude readout via lock-in detector SR7265:

def nA2lockin(
    S = 1.0,                # sensitivity in units of Volt
    R = 1.0e4,              # measurement resistor to cold-ground
    A = 1.0e4,              # amplification of measured AC signal
    C = 2*np.sqrt(2)/np.pi, # conversion factor of the lock-in detector with square-input signal
    M = 10.0,               # magnitude of detected voltage on lock-in at maximal sensitivity
    ):
    """ Function to convert Nanoampere to Lock-In units"""
    return M*C*A*R/1e9/S

i0 = controls.add('i0',
            source=lockins[0].R,
            label = r'Lock-in current $I_{\rm LI}$',
            unit = 'nA',
            scale = nA2lockin(S=lockins[0].sensitivity()),
            )

# Phase readout via lock-in detector SR7265:

p0 = controls.add('p0',
            source=lockins[0].P,
            label = r'Lock-in phase $\phi_{\rm LI}$',
            unit = lockins[0].P.unit, # degree
            )

In [ ]:
progress.update(1)

### Example magnetic field control via ADC analog-output and Bouhnik

In [ ]:
# Magnetic field control:

# The scale factor is defined via
#  1. the maximum is +- 12 A for +- 10 V input voltage on the Bouthnik current source and
#  2. the magnetic field response of the coil that is 52.6265 mT/A on Wodan.
mT2Vadc = 1/1.2/52.6265
lim = 10.0/mT2Vadc # Limits in Millitesla defined ADC limit of +-10 V

b_field = controls.add('b_field',
            source=adc.ao0.v,
            label = r'Magnetic Field $B$',
            unit = 'mT',
            scale = mT2Vadc,
            step = 0.1, # Max. step size in "user-units" (scale; here: mT)
            inter_delay = 0.001, # Delay between steps in seconds.
            vals  = vals.Numbers(-lim,lim),
            # IMPORTANT: Do not use the initial_value parameter for initialisation!!!
            )

In [ ]:
progress.update(1)

# 3. Fast sequence controls

In [ ]:
# TO DO

# 4. Configuration
### Summary
Let us first get an overview on the controls we have defined:

In [ ]:
controls.markdown()

We can also look on the controls in form of a dictionary:

In [ ]:
controls.dictionary()

### Set control values
Modifying the aforeshown dictionary, you can rapidly adapt the values on your will and apply them:

In [ ]:
controls.apply(
    {
        'qpc':       -1.5    ,
        'barrier':   -1.0    ,
        'dc_bias':    1.0e-3 ,
        'ac_freq':  666.0    ,
        'ac_ampl':    100e-6 ,
        'ac_chan':   [1,0]   ,
        'ac_bias':   False   ,
        'b_field':    0.0    ,
    }
)
controls.markdown()

In [ ]:
adc.measure()

In [ ]:
controls.qpc()